## Категоризация названий товаров

### Данные для решения задачи

```
demo/data
├── categories.csv
└── products.csv
```

**categories.csv** - файл с категориями на маркетплейсе. У каждой категории есть id, заголовок и путь в дереве
категорий.

Допустим у категории 2642 заголовок "Мелкие инструменты", а путь в дереве категорий - `1.10016.10072.10690.2642`. Если
заменить id категорий в этом пути на заголовки, то получим следующее дерево:

* Строительство и ремонт
    * Ручной инструмент и оснастка
        * Столярно-слесарные инструменты
            * Мелкие инструменты

**products.csv** - файл с товарами на маркетплейсе. У каждого товара есть id, заголовок и id категории, которой он
принадлежит. Товар всегда принадлежит листовой категории в дереве.

### Моделирование и оценка

Категоризация названий товаров это процесс определения категории на основе текстовых данных товара. Фактически, нужно
решить задачу классификации с множеством классов на основе текстового ввода. Для тестового задания мы упростили задачу и
ограничились 3 большими категориям и взяли лишь часть товаров.

Для того, чтобы решить задачу предсказания `category_id` по `product_title` можно использовать любую модель и любой
препроцессинг, если он подходит для этой задачи. Если вы будете использовать несколько моделей, то будет замечательно,
если вы приведете аргументацию своего выбора.

### План действий:

1. Провести предобработку данных;
2. Выполнить токенизацию текста, то есть разбить его  на слова;
3. Лемматизация: приведение к начальной словарной форме;
4. Очистить текст от стоп-слов и ненужных символов;
5. На выходе у каждого исходного текста образуется свой список токенов;
6. Токены передают модели, которая переводит их в векторные представления. Для этого модель обращается к составленному заранее словарю токенов. На выходе для каждого текста образуются векторы заданной длины;
7. На финальном этапе модели передаются признаки (векторы). И она определяет категорию.

### Преобработка, очистка данных, лемматизация

In [2]:
! pip install pymystem3

In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC

In [4]:
data_cat = pd.read_csv('./demo/data/categories.csv')
print(data_cat.head(10))
print('===============================')
display(data_cat.info())
print('===============================')
data_cat.describe()
print(data_cat.isnull().sum()) # Проверяем на наличие пропусков
print ('Дубликатов в таблице:', data_cat.duplicated().sum()) # Проверяем на наличие дубликатов

   category_id                  category_title              category_path
0        13021                     Базы и топы        1.10012.10113.13021
1         2740               Полки и подставки   1.10018.10110.12842.2740
2        13182                          Салюты  1.10018.10118.10749.13182
3         2864                 Ложки для обуви   1.10018.10110.12823.2864
4        14154     Аромабудильники и картриджи        1.10020.10227.14154
5        12486                        Травяные  1.10018.10049.10770.12486
6        13419                         Ножницы  1.10012.10113.10355.13419
7         2736  Аксессуары для стирки и глажки   1.10018.10110.12823.2736
8        14255             Охлаждение напитков  1.10018.10115.11097.14255
9        10483               Пледы и покрывала        1.10018.10049.10483
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  -

None

category_id       0
category_title    0
category_path     0
dtype: int64
Дубликатов в таблице: 0


In [5]:
data_prod = pd.read_csv('./demo/data/products.csv')
print(data_prod.head(10))
print('===============================')
display(data_prod.info())
print('===============================')
data_prod.describe()
print(data_prod.isnull().sum()) # Проверяем на наличие пропусков
print ('Дубликатов в таблице:', data_prod.duplicated().sum()) # Проверяем на наличие дубликатов

   row_number                              product_title  category_id
0           1         Термокружка с животными 350/500 мл        12407
1           2      Пластиковая емкость для хранения круп        12667
2           3    Контейнер с дозатором для хранения круп        13901
3           4                 Контейнер для хранения яиц        13674
4           5                             Губкодержатель        13254
5           6                Глубокая тарелка для овощей        11999
6           7  Держатель на кухонный гарнитур под отходы        14095
7           8                       Крючок для полотенец        13170
8           9           Крючок для полотенца на присоске         2744
9          10                               Мерная ложка        11978
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144082 entries, 0 to 144081
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   row_number     144082 non-n

None

row_number       0
product_title    0
category_id      0
dtype: int64
Дубликатов в таблице: 0


In [6]:
merge_data = data_prod.join(data_cat.set_index('category_id'), on='category_id', how='left') # Объединим обе таблицы в одну для удобства
merge_data.describe()
merge_data = merge_data.head(50000)
#display(merge_data.head(20))
merge_data.category_id.value_counts() # Посмотрим баланс классов и узнаем количество уникальных категорий

11937    6757
13408    4228
12171    1715
14241    1181
13407    1042
         ... 
13738       1
12568       1
13506       1
12696       1
12436       1
Name: category_id, Length: 660, dtype: int64

In [7]:
m = Mystem()
corpus = merge_data['product_title'].values.astype('U')

def lemmatize(text): # лемматизируем слова
    lemma = m.lemmatize(text) 
    return "".join(lemma) 

def clear_text(text):
    clear_1 = re.sub(r'[^а-яА-ЯёЁ ]', ' ', text) # очищаем леммы от ненужных символов
    clear_2 = " ".join(clear_1.split())
    return clear_2


print("Исходный текст:", corpus[10])
print("Очищенный и лемматизированный текст:", lemmatize(clear_text(corpus[10])))

result = []

for text in corpus:
    result_lemma = lemmatize(clear_text(text))
    result.append(result_lemma)


merge_data['lemm_text'] = pd.DataFrame(result) # Результат лемматизации запишем в столбец 'lemm_text'
 
print(merge_data['lemm_text'].head(10))

Исходный текст: Держатель для мусорного ведра
Очищенный и лемматизированный текст: держатель для мусорный ведро

0                    термокружок с животное мл\n
1       пластиковый емкость для хранение крупа\n
2       контейнер с дозатор для хранение крупа\n
3                  контейнер для хранение яйцо\n
4                               губкодержатель\n
5                    глубокий тарелка для овощ\n
6    держатель на кухонный гарнитур под отходы\n
7                         крючок для полотенце\n
8             крючок для полотенце на присосок\n
9                                 мерный ложка\n
Name: lemm_text, dtype: object


In [8]:
train, test = train_test_split(merge_data, test_size=0.25, random_state=12345) # Разделим данные на тренировочную и тестовую выборки

corpus_train = train['lemm_text'].values.astype('U')
corpus_test = test['lemm_text'].values.astype('U')

In [9]:
# Посмотрим, как часто уникальное слово встречается во всём корпусе и в отдельном его тексте.
# Оценка важности слова определяется величиной TF-IDF. 
# IDF нужна в формуле, чтобы уменьшить вес слов, наиболее распространённых в любом тексте заданного корпуса.
count_tf_idf = TfidfVectorizer(max_features=3000) # Ограничим количество фичей для ускорения процесса обработки данных
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
df_train = pd.DataFrame(tf_idf_train.toarray(), columns=count_tf_idf.get_feature_names())
train = pd.concat([train.reset_index(), df_train.reset_index()], axis=1)
df_train = [] # обнуление ради высвобождения RAM

tf_idf_test = count_tf_idf.transform(corpus_test)
df_test = pd.DataFrame(tf_idf_test.toarray(), columns=count_tf_idf.get_feature_names())
test = pd.concat([test.reset_index(), df_test.reset_index()], axis=1)
df_test = [] # обнуление ради высвобождения RAM

corpus = [] # обнуление ради высвобождения RAM
corpus_test = [] # обнуление ради высвобождения RAM
# Большая величина TF-IDF говорит об уникальности слова в тексте по отношению к корпусу. 
# Чем чаще оно встречается в конкретном тексте и реже в остальных, тем выше значение TF-IDF.

In [10]:
features_train = train.drop(['category_title', 'category_path', 'category_id', 'row_number', 'lemm_text', 'product_title', 'index'], axis=1)
target_train = train['category_id']
target_test = test['category_id']
features_test = test.drop(['category_title', 'category_path', 'category_id', 'row_number', 'lemm_text', 'product_title','index'], axis=1)

In [11]:
print(features_train.head(10))
print(target_train.head(10))

    аа  ааа  абразив  абразивность  абразивный  абрикосовый  абстракция  ава  \
0  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
1  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
2  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
3  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
4  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
5  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
6  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
7  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
8  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   
9  0.0  0.0      0.0           0.0         0.0          0.0         0.0  0.0   

   авокадо  авто  ...  яйцерезка  яйцо  японский  ярата  яркий  яркость  ярус  \
0      0.0   0.0  ...        0.0   0.0

### Обучение моделей

In [13]:
# Создадим таблицу, куда будем записывать имя и качество модели
research_result = pd.DataFrame(columns=['Model_name', 'F1-score', 'Final cross-val score'])
# Создадим функцию для обучения, предсказания и исследования качетсва модели
def train_model(model, name, features_train, target_train, features_test, target_test):
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)
    f1 = f1_score(predictions, target_test, average='weighted') # Укажем аргумент 'weighted', так как классы несбалансированы
    cv_score = cross_val_score(model, features_train, target_train, cv=3) # Посчитаем оценки, вызвав функцию cross_value_score с тремя блоками 
    final_cv_score =np.sum(cv_score) / len(cv_score)
    results = pd.DataFrame([[name, f1, final_cv_score]], columns=['Model_name', 'F1-score', 'Final cross-val score'])
    return results, predictions

In [14]:
# Логистическая регрессия
model_regression=LogisticRegression(random_state=12345, solver='liblinear')
model_regression_results, model_regression_predictions = train_model(model_regression, 'Logistic Regression', features_train, target_train, features_test, target_test)
research_result = research_result.append(model_regression_results)
print(research_result)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


            Model_name  F1-score  Final cross-val score
0  Logistic Regression  0.806229               0.757387


In [15]:
# Дерево решений
model_tree=DecisionTreeClassifier(random_state=12345)
#(criterion = "entropy", max_depth = 12,  min_samples_split = 4, min_samples_leaf = 1, random_state=12345)
model_tree_results, model_tree_predictions = train_model(model_tree, 'Decision Tree Classifier', features_train, target_train, features_test, target_test)
research_result = research_result.append(model_tree_results)
print(research_result)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


                 Model_name  F1-score  Final cross-val score
0       Logistic Regression  0.806229               0.757387
0  Decision Tree Classifier  0.773604               0.755333


In [16]:
# Cлучайный лес
model_forest = RandomForestClassifier(n_estimators=16, max_depth=25, min_samples_leaf=1, random_state=12345)
model_forest_results, model_forest_predictions = train_model(model_forest, 'Random Forest Classifier', features_train, target_train, features_test, target_test)
research_result = research_result.append(model_forest_results)
print(research_result)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


                 Model_name  F1-score  Final cross-val score
0       Logistic Regression  0.806229               0.757387
0  Decision Tree Classifier  0.773604               0.755333
0  Random Forest Classifier  0.607823               0.551840


In [17]:
# Классификатор LightGBM
model_LGBMR = LGBMClassifier(boosting_type='gbdt', random_state=12345)
#, num_leaves=4, max_depth=20, learning_rate=0.3, n_estimators=10, random_state=12345)
model_LGBMR_results, model_LGBMR_predictions = train_model(model_LGBMR, 'LGBM Classifier', features_train, target_train, features_test, target_test)
research_result = research_result.append(model_LGBMR_results)
print(research_result)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


                 Model_name  F1-score  Final cross-val score
0       Logistic Regression  0.806229               0.757387
0  Decision Tree Classifier  0.773604               0.755333
0  Random Forest Classifier  0.607823               0.551840
0           LGBM Classifier  0.015898               0.038293


In [18]:
# Классификация линейных опорных векторов
model_svc = LinearSVC(C=3, random_state=12345)
model_svc_results, model_svc_predictions = train_model(model_svc, 'Linear SVC', features_train, target_train, features_test, target_test)
research_result = research_result.append(model_svc_results)
print(research_result)
print(model_svc_predictions)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


                 Model_name  F1-score  Final cross-val score
0       Logistic Regression  0.806229               0.757387
0  Decision Tree Classifier  0.773604               0.755333
0  Random Forest Classifier  0.607823               0.551840
0           LGBM Classifier  0.015898               0.038293
0                Linear SVC  0.806427               0.793627
[12171 14235 13408 ... 13954 13485 12529]


In [19]:
print(research_result)

                 Model_name  F1-score  Final cross-val score
0       Logistic Regression  0.806229               0.757387
0  Decision Tree Classifier  0.773604               0.755333
0  Random Forest Classifier  0.607823               0.551840
0           LGBM Classifier  0.015898               0.038293
0                Linear SVC  0.806427               0.793627


### Выводы:

* Исходя из задачи и данных я выбрала метрику F1-score (среднее гармоническое точности и полноты). Так как Accuracy в нашем случае не подходит в виду дисбаланса классов.
* В качестве дополнительной оценки использовала "среднюю оценку качества модели", рассчитанную с помощью кросс-валидации.
* Для более быстрого получения результатов работы модели запускались на 1/3 части всех данных. 
* Из всех рассмотренных мной моделей наиболее высоких метрик смогли достигнуть две модели - это Linear Support Vector Classification (0.80) и Логистическая регрессия (0.80). Дерево решений показало неплохую метрику (0.77) Случайный лес и  LGBM-модель - их метрики были неудовлетворительными, однако, возможно, просто нужно подобрать подходящие гиперпараметры. Также запуск моделей на полном объеме данных явно повысит метрики. Таким образом, для демо я выбрала модель Linear Support Vector Classification, как модель с сочетанием наиболее высоких показателей обеих метрик (как F1-score, так и Final cross-val score).

#### Изучение примеров верно и неверно категоризированных товаров:
* Тест 1. Вы ввели:  Защитное стекло для Xiaomi Mi 8
Определена категория:
     category_id            category_title              category_path
        13408  Защитные стекла и пленки  1.10020.10044.10398.13408 (верно)
* Тест 2. Вы ввели:  Футляр для зубной щётки
Определена категория:
     category_id        category_title             category_path
        2741  Диспенсеры, дозаторы  1.10018.10110.12842.2741 (верно)
* Тест 3. Вы ввели:  Чайник заварочный 1 литр
Определена категория:
     category_id      category_title              category_path
        13027  Заварочные чайники  1.10018.10115.11519.13027 (и верно, и неверно)
Однако по данным из таблицы, запрашиваемый мною чайник относится к категории 14019,Чайники,1.10018.10115.11519.14019.
Модель верно определила категорию запрашиваемого товара. Однако, возможно, сам товар был помещён в неверную категорию. Так как мы ищем конкретно чайник заварочный, ему следовало бы располагаться в категории заварочных чайников :) 
* Тест 4. Вы ввели:  Губкодержатель
Определена категория:
     category_id         category_title             category_path
         2804  Прочая электротехника  1.10020.10074.10641.2804 (неверно)
* Тест 5. Вы ввели:  Бенгальские свечи
Определена категория:
     category_id         category_title             category_path
         2804  Прочая электротехника  1.10020.10074.10641.2804 (неверно)
Есть предположение, что товары, примеров которых для обучения модели было мало (малое число по сравнению с другими классами), неверно определяются и попадают в категорию Прочая электротехника. Возможно эта категория находится внизу "дерева решений".          

In [20]:
from joblib import dump

dump(model_svc, 'model_svc.joblib')
dump(count_tf_idf, 'vectorizer.joblib')

['vectorizer.joblib']